#Graphs

In [ ]:
def edges(n):
    E = []
    for i in range(n):
        E.append([i+1, -i-1])
        for j in range(i+1, n):
            E.append([i+1, j+1])
    return E

In [ ]:
def perm(n):
    if n == 1: return [[False], [True]]
    P = perm(n-1)
    per = []
    for p in P:
        per.append([False]+p)
    for p in P:
        per.append([True]+p)
    return per

In [ ]:
def is_cubic(n, g):
    for i in range(n):
        num = 0
        for e in g:
            if i+1 in e:
                num += 1
        if num != 3: return False
    return True

In [ ]:
def cubic(n):
    E = edges(n)
    l = len(E)
    P = perm(l)
    G = []
    for p in P:
        g = []
        for i in range(l):
            if p[i]: g.append(E[i])
        if is_cubic(n, g): G.append(g)
    return G

In [ ]:
def to_list(g, n):
    graph = []
    #print(g)
    for i in range(n):
        graph.append([])
    for e in g:
        if e[0] > 0: graph[e[0]-1].append(e[1])
        if e[1] > 0: graph[e[1]-1].append(e[0])
    return graph

#Matchings

In [ ]:
def matchings(g, i):
    if i == len(g)-1:
        return [[], [g[i]]]
    M = []
    m = [g[i]]
    perms = matchings(g, i+1)
    for p in perms:
        M.append(p)
    for p in perms:
        M.append(m + p)
    return M

In [ ]:
def is_perfect(m, n):
    for i in range(n):
        #print(i+1)
        count = 0
        for e in m:
            #print(e)
            if i+1 in e:
                count += 1
        if count != 1: return False
    return True

In [ ]:
def perfect_matchings(g, n):
    M = matchings(g, 0)
    PM = []
    for m in M:
        if is_perfect(m, n): PM.append(m)
    return PM

#Cycles

In [ ]:
def in_s(s, v1, v2):
    if ([v1, v2] in s) or ([v2, v1] in s): return 1
    else: return -1

In [ ]:
def alt_c(g, m, s, cur, prev_in_m, c_v, c_e):
    paths = []
    for v in g[cur-1]:
        #print(cur, v)
        if v not in c_v:
            if in_s(c_e, cur, v) < 0:
                check = in_s(m, cur, v)
                if prev_in_m != in_s(m, cur, v):
                    if v == s:
                        paths.append([[cur, v]])
                    else:
                        new_c_e = c_e.copy()
                        new_c_e = new_c_e + [[cur, v]]
                        P = alt_c(g, m, s, v, check, c_v, new_c_e)
                        for p in P:
                            paths.append([[cur, v]]+p)
                #else: print("prev_in_m == in_s")
            #else: print("[cur, v] in c_e")
        #else: print("v in c_v")
    return paths

In [ ]:
def cycles(g, m, n):
    l = len(g)
    c_v = [-i for i in range(l*2)]
    cycles = []
    for i in range(n):
        c = alt_c(g, m, i, i, 0, c_v, [])
        c_v.append(i)
        cycles += c
    return cycles

In [ ]:
def check_alternating(cycle, m):
    n = len(cycle)
    if n < 4:
        #print("E1", n)
        return False
    if n % 2 != 0:
        #print("E2", n)
        return False
    check = cycle[0] in m
    for i in range(1, n):
        c = (cycle[i] in m or [cycle[i][1], cycle[i][0]] in m)
        if c == check:
            #print("E3", i)
            return False
        check = c
    return True

In [ ]:
C = cubic(4)

In [ ]:
PM = perfect_matchings(C[0], 4)

In [ ]:
PM[0]

[[1, 4], [2, 3]]

In [ ]:
Cy = cycles(to_list(C[0], 4), PM[0], 4)
for c in Cy:
    if check_alternating(c, PM[0]):
        print(c)

[[1, 2], [2, 3], [3, 4], [4, 1]]
[[1, 3], [3, 2], [2, 4], [4, 1]]
[[1, 4], [4, 2], [2, 3], [3, 1]]
[[1, 4], [4, 3], [3, 2], [2, 1]]


In [ ]:
def is_disjoint(c1, c2):
    set1 = set()
    set2 = set()
    for e in c1:
        set1.add(e[0])
        set1.add(e[1])
    for e in c2:
        set2.add(e[0])
        set2.add(e[1])
    for v in set2:
        if v in set1: return False
    return True

In [ ]:
def possible_add(C, cycle):
    for c in C:
        if not is_disjoint(c, cycle): return False
    return True

In [ ]:
def max_disjoint(max, C):
    if len(C) == 0: return max
    if possible_add(max, C[0]):
        v1 = max_disjoint(max+[C[0]], C[1:])
        v2 = max_disjoint(max, C[1:])
        if len(v1) > len(v2): return v1
        else: return v2
    else: return max_disjoint(max, C[1:])

In [ ]:
def m_alternating_cycles(graph, m, n):
    g = to_list(graph, n)
    C = cycles(g, m, n)
    ans = []
    for c in C:
        if check_alternating(c, m):
            #print(c)
            ans.append(c)
    return max_disjoint([], ans)

In [ ]:
def m_cycles_dist(g, n):
    M = perfect_matchings(g, n)
    dist = []
    for m in M:
        dist.append([m, len(m_alternating_cycles(g, m, n))])
    return(dist)

#LP

In [ ]:
!pip install scipy
!pip install --upgrade scipy

     |████████████████████████████████| 38.1 MB 1.5 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
from scipy.optimize import linprog

In [ ]:
def edge_in_s(e, S):
    if (e in S) or ([e[1], e[0]] in S): return True
    return False


In [ ]:
def solve_lp_burl(g, n):
    dist = m_cycles_dist(g, n)

    c = [dist[i][1] for i in range(len(dist))]
    bounds = [(0, 1) for i in range(len(dist))]

    v1 = [1 for i in range(len(dist))]
    A_eq = []
    A_eq.append(v1)
    for i in range(len(g)):
        vi = []
        for elem in dist:
            if g[i] in elem[0]:
                vi.append(1)
            else: vi.append(0)
        #print(vj)
        A_eq.append(vi)
    
    b_eq = [1/3 for i in range(len(A_eq))]
    b_eq[0] = 1

    res = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=bounds)
    return res

#AUX

In [ ]:
def next_perm(l):
    i = len(l)-1
    while i > 0:
        if l[i]:
            l[i] = not l[i]
            i -= 1
        else:
            l[i] = not l[i]
            return l, False
    if l[i]:
        return l, True
    else:
        l[i] = not l[i]
        return l, False

In [ ]:
from IPython.display import clear_output

In [ ]:
N = 5

E = edges(N)

count1 = 0
count2 = 0

stop = False

g = [False for i in range(len(E))]

while not stop:
    graph = []
    for i in range(len(E)):
        if g[i]: graph.append(E[i])
    if is_cubic(N, graph):
        res = solve_lp_burl(graph, N)
        if res.fun >= 1/3:
            print(res.fun)
            print(graph)
            count1 += 1
        else: count2 += 1
    g, stop = next_perm(g)
#clear_output(True)
print(count1, count2)  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.


1.0000000000077924
[[1, 3], [1, 4], [1, 5], [2, 3], [2, 4], [2, 5], [3, -3], [4, -4], [5, -5]]
0.3333333358333333
[[1, 3], [1, 4], [1, 5], [2, -2], [2, 4], [2, 5], [3, -3], [3, 5], [4, -4]]
0.3333333358333333
[[1, 3], [1, 4], [1, 5], [2, -2], [2, 4], [2, 5], [3, -3], [3, 4], [5, -5]]
0.3333333358333333
[[1, 3], [1, 4], [1, 5], [2, -2], [2, 3], [2, 5], [3, 4], [4, -4], [5, -5]]
0.3333333358333333
[[1, 3], [1, 4], [1, 5], [2, -2], [2, 3], [2, 5], [3, -3], [4, -4], [4, 5]]
0.3333333358333333
[[1, 3], [1, 4], [1, 5], [2, -2], [2, 3], [2, 4], [3, 5], [4, -4], [5, -5]]
0.3333333358333333
[[1, 3], [1, 4], [1, 5], [2, -2], [2, 3], [2, 4], [3, -3], [4, 5], [5, -5]]
0.3333333358333333
[[1, 2], [1, 4], [1, 5], [2, 3], [2, 5], [3, -3], [3, 4], [4, -4], [5, -5]]
0.3333333358333333
[[1, 2], [1, 4], [1, 5], [2, 3], [2, 4], [3, -3], [3, 5], [4, -4], [5, -5]]
0.3333333358333333
[[1, 2], [1, 4], [1, 5], [2, -2], [2, 5], [3, -3], [3, 4], [3, 5], [4, -4]]
0.3333333358333333
[[1, 2], [1, 4], [1, 5], [2, -2

47 min 15 sec on 7 vertex graph

In [ ]:
n = 5
C = cubic(n)
for c in C:
    res = solve_lp_burl(c, n)
    if res.fun >= 1/3:
        #print(res.fun)
        #print(c)
        count1 += 1
    else: count2 += 1
print(count1, count2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.


140 84


In [ ]:
G5 = [[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 10], [9, 13], [10, 11], [10, 12], [11, 14], [12, 13], [12, 15], [13, 16], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 10], [9, 13], [10, 11], [10, 12], [11, 15], [12, 13], [12, 14], [13, 16], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 10], [9, 13], [10, 11], [10, 12], [11, 14], [12, 13], [12, 16], [13, 15], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 10], [9, 13], [10, 11], [10, 12], [11, 14], [12, 13], [12, 15], [13, 16], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 10], [9, 13], [10, 11], [10, 12], [11, 15], [12, 13], [12, 14], [13, 16], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 10], [9, 13], [10, 11], [10, 12], [11, 14], [12, 13], [12, 16], [13, 15], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 10], [9, 13], [10, 11], [10, 12], [11, 14], [12, 13], [12, 15], [13, 16], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 10], [9, 13], [10, 11], [10, 12], [11, 15], [12, 13], [12, 14], [13, 16], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 10], [9, 13], [10, 11], [10, 12], [11, 14], [12, 13], [12, 16], [13, 15], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 12], [9, 10], [10, 11], [10, 14], [11, 13], [12, 13], [12, 15], [13, 16], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 12], [9, 10], [10, 11], [10, 15], [11, 13], [12, 13], [12, 14], [13, 16], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 12], [9, 10], [10, 11], [10, 14], [11, 13], [12, 13], [12, 16], [13, 15], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 12], [9, 10], [10, 11], [10, 14], [11, 13], [12, 13], [12, 15], [13, 16], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 12], [9, 10], [10, 11], [10, 15], [11, 13], [12, 13], [12, 14], [13, 16], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 12], [9, 10], [10, 11], [10, 14], [11, 13], [12, 13], [12, 16], [13, 15], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 12], [9, 10], [10, 11], [10, 14], [11, 13], [12, 13], [12, 15], [13, 16], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 12], [9, 10], [10, 11], [10, 15], [11, 13], [12, 13], [12, 14], [13, 16], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 12], [9, 10], [10, 11], [10, 14], [11, 13], [12, 13], [12, 16], [13, 15], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 12], [9, 10], [10, 11], [10, 13], [11, 14], [12, 13], [12, 15], [13, 16], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 12], [9, 10], [10, 11], [10, 13], [11, 15], [12, 13], [12, 14], [13, 16], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 12], [9, 10], [10, 11], [10, 13], [11, 14], [12, 13], [12, 16], [13, 15], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 12], [9, 10], [10, 11], [10, 13], [11, 14], [12, 13], [12, 15], [13, 16], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 12], [9, 10], [10, 11], [10, 13], [11, 15], [12, 13], [12, 14], [13, 16], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 12], [9, 10], [10, 11], [10, 13], [11, 14], [12, 13], [12, 16], [13, 15], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 12], [9, 10], [10, 11], [10, 13], [11, 14], [12, 13], [12, 15], [13, 16], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 12], [9, 10], [10, 11], [10, 13], [11, 15], [12, 13], [12, 14], [13, 16], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 12], [9, 10], [10, 11], [10, 13], [11, 14], [12, 13], [12, 16], [13, 15], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, 14], [11, 13], [12, 13], [12, 15], [13, 16], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, 15], [11, 13], [12, 13], [12, 14], [13, 16], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, 14], [11, 13], [12, 13], [12, 16], [13, 15], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, 14], [11, 13], [12, 13], [12, 15], [13, 16], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, 15], [11, 13], [12, 13], [12, 14], [13, 16], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, 14], [11, 13], [12, 13], [12, 16], [13, 15], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, 14], [11, 13], [12, 13], [12, 15], [13, 16], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, 15], [11, 13], [12, 13], [12, 14], [13, 16], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, 14], [11, 13], [12, 13], [12, 16], [13, 15], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 6], [5, 8], [6, 7], [6, 10], [7, 9], [8, 9], [8, 11], [9, 13], [10, 11], [10, 14], [11, 12], [12, 13], [12, 15], [13, 16], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 6], [5, 8], [6, 7], [6, 10], [7, 9], [8, 9], [8, 11], [9, 13], [10, 11], [10, 15], [11, 12], [12, 13], [12, 14], [13, 16], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 6], [5, 8], [6, 7], [6, 10], [7, 9], [8, 9], [8, 11], [9, 13], [10, 11], [10, 14], [11, 12], [12, 13], [12, 16], [13, 15], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 6], [5, 8], [6, 7], [6, 10], [7, 9], [8, 9], [8, 11], [9, 13], [10, 11], [10, 14], [11, 12], [12, 13], [12, 15], [13, 16], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 6], [5, 8], [6, 7], [6, 10], [7, 9], [8, 9], [8, 11], [9, 13], [10, 11], [10, 15], [11, 12], [12, 13], [12, 14], [13, 16], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 6], [5, 8], [6, 7], [6, 10], [7, 9], [8, 9], [8, 11], [9, 13], [10, 11], [10, 14], [11, 12], [12, 13], [12, 16], [13, 15], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 6], [5, 8], [6, 7], [6, 10], [7, 9], [8, 9], [8, 11], [9, 13], [10, 11], [10, 14], [11, 12], [12, 13], [12, 15], [13, 16], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 6], [5, 8], [6, 7], [6, 10], [7, 9], [8, 9], [8, 11], [9, 13], [10, 11], [10, 15], [11, 12], [12, 13], [12, 14], [13, 16], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 6], [5, 8], [6, 7], [6, 10], [7, 9], [8, 9], [8, 11], [9, 13], [10, 11], [10, 14], [11, 12], [12, 13], [12, 16], [13, 15], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 10], [7, 9], [8, 9], [8, 12], [9, 11], [10, 11], [10, 13], [11, 14], [12, 13], [12, 15], [13, 16], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 10], [7, 9], [8, 9], [8, 12], [9, 11], [10, 11], [10, 13], [11, 15], [12, 13], [12, 14], [13, 16], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 10], [7, 9], [8, 9], [8, 12], [9, 11], [10, 11], [10, 13], [11, 14], [12, 13], [12, 16], [13, 15], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 10], [7, 9], [8, 9], [8, 12], [9, 11], [10, 11], [10, 13], [11, 14], [12, 13], [12, 15], [13, 16], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 10], [7, 9], [8, 9], [8, 12], [9, 11], [10, 11], [10, 13], [11, 15], [12, 13], [12, 14], [13, 16], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 10], [7, 9], [8, 9], [8, 12], [9, 11], [10, 11], [10, 13], [11, 14], [12, 13], [12, 16], [13, 15], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 10], [7, 9], [8, 9], [8, 12], [9, 11], [10, 11], [10, 13], [11, 14], [12, 13], [12, 15], [13, 16], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 10], [7, 9], [8, 9], [8, 12], [9, 11], [10, 11], [10, 13], [11, 15], [12, 13], [12, 14], [13, 16], [14, -14], [16, -16], [14, 15], [15, 16]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 10], [7, 9], [8, 9], [8, 12], [9, 11], [10, 11], [10, 13], [11, 14], [12, 13], [12, 16], [13, 15], [14, -14], [16, -16], [14, 15], [15, 16]]]

In [ ]:
G6 = [[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, 13], [11, 14], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 10], [9, 13], [10, 11], [10, 12], [11, 14], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, 14], [11, 13], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, 13], [11, 14], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 10], [9, 13], [10, 11], [10, 12], [11, 14], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, 14], [11, 13], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, 13], [11, 14], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 10], [9, 13], [10, 11], [10, 12], [11, 14], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, 14], [11, 13], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, 13], [11, 14], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 10], [9, 13], [10, 11], [10, 12], [11, 14], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, 14], [11, 13], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, 13], [11, 14], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 10], [9, 13], [10, 11], [10, 12], [11, 14], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, 14], [11, 13], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, 13], [11, 14], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 10], [9, 13], [10, 11], [10, 12], [11, 14], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, 14], [11, 13], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 6], [5, 8], [6, 7], [6, 10], [7, 9], [8, 9], [8, 11], [9, 12], [10, 11], [10, 13], [11, 14], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 6], [5, 8], [6, 7], [6, 10], [7, 9], [8, 9], [8, 11], [9, 13], [10, 11], [10, 12], [11, 14], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 6], [5, 8], [6, 7], [6, 10], [7, 9], [8, 9], [8, 11], [9, 12], [10, 11], [10, 14], [11, 13], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 6], [5, 8], [6, 7], [6, 10], [7, 9], [8, 9], [8, 11], [9, 12], [10, 11], [10, 13], [11, 14], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 6], [5, 8], [6, 7], [6, 10], [7, 9], [8, 9], [8, 11], [9, 13], [10, 11], [10, 12], [11, 14], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 6], [5, 8], [6, 7], [6, 10], [7, 9], [8, 9], [8, 11], [9, 12], [10, 11], [10, 14], [11, 13], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 6], [5, 8], [6, 7], [6, 10], [7, 9], [8, 9], [8, 11], [9, 12], [10, 11], [10, 13], [11, 14], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 6], [5, 8], [6, 7], [6, 10], [7, 9], [8, 9], [8, 11], [9, 13], [10, 11], [10, 12], [11, 14], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 6], [5, 8], [6, 7], [6, 10], [7, 9], [8, 9], [8, 11], [9, 12], [10, 11], [10, 14], [11, 13], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 10], [7, 9], [8, 9], [8, 12], [9, 11], [10, 11], [10, 13], [11, 14], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 10], [7, 9], [8, 9], [8, 13], [9, 11], [10, 11], [10, 12], [11, 14], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 10], [7, 9], [8, 9], [8, 12], [9, 11], [10, 11], [10, 14], [11, 13], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 10], [7, 9], [8, 9], [8, 12], [9, 11], [10, 11], [10, 13], [11, 14], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 10], [7, 9], [8, 9], [8, 13], [9, 11], [10, 11], [10, 12], [11, 14], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 10], [7, 9], [8, 9], [8, 12], [9, 11], [10, 11], [10, 14], [11, 13], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 10], [7, 9], [8, 9], [8, 12], [9, 11], [10, 11], [10, 13], [11, 14], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 10], [7, 9], [8, 9], [8, 13], [9, 11], [10, 11], [10, 12], [11, 14], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 10], [7, 9], [8, 9], [8, 12], [9, 11], [10, 11], [10, 14], [11, 13], [12, -12], [14, -14], [12, 13], [13, 14]]]

In [ ]:
for g in G5:
    res = solve_lp_burl(g, 11)
    print(res.fun)
    if res.fun < 1/3:
        print(g)
        #print(g)

1


In [ ]:
for g in G6:
    res = solve_lp_burl(g, 14)
    if res.fun < 1/3:
        print(res.fun)
        print(g)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: OptimizeWarning: Solving system with option 'cholesky':True failed. It is normal for this to happen occasionally, especially as the solution is approached. However, if you see this frequently, consider setting option 'cholesky' to False.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: OptimizeWarning: Solving system with option 'sym_pos':True failed. It is normal for this to happen occasionally, especially as the solution is approached. However, if you see this frequently, consider setting option 'sym_pos' to False.
/usr/local/lib/python3.7/dist-packages/scipy/optimize/_linprog_ip.py:117: LinAlgWarning: Ill-conditioned matrix (rcond=1.35345e-18): result may not be accurate.
  return sp.linalg

In [ ]:
G4 = [[[-1, 1], [1, 2], [1, 3], [-5, 5], [2, 3], [2, 4], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 10], [8, 9], [8, 11], [9, 12], [10, -10], [12, -12], [10, 11], [11, 12]],
[[-1, 1], [1, 2], [1, 3], [-5, 5], [2, 3], [2, 4], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 10], [9, 12], [10, -10], [12, -12], [10, 11], [11, 12]],
[[-1, 1], [1, 2], [1, 3], [-5, 5], [2, 3], [2, 4], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 10], [8, 9], [8, 12], [9, 11], [10, -10], [12, -12], [10, 11], [11, 12]],
[[-1, 1], [1, 2], [-3, 3], [1, 5], [2, 3], [2, 4], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 10], [8, 9], [8, 11], [9, 12], [10, -10], [12, -12], [10, 11], [11, 12]],
[[-1, 1], [1, 2], [-3, 3], [1, 5], [2, 3], [2, 4], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 10], [9, 12], [10, -10], [12, -12], [10, 11], [11, 12]],
[[-1, 1], [1, 2], [-3, 3], [1, 5], [2, 3], [2, 4], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 10], [8, 9], [8, 12], [9, 11], [10, -10], [12, -12], [10, 11], [11, 12]],
[[-1, 1], [-2, 2], [1, 3], [1, 5], [2, 3], [2, 4], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 10], [8, 9], [8, 11], [9, 12], [10, -10], [12, -12], [10, 11], [11, 12]],
[[-1, 1], [-2, 2], [1, 3], [1, 5], [2, 3], [2, 4], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 10], [9, 12], [10, -10], [12, -12], [10, 11], [11, 12]],
[[-1, 1], [-2, 2], [1, 3], [1, 5], [2, 3], [2, 4], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 10], [8, 9], [8, 12], [9, 11], [10, -10], [12, -12], [10, 11], [11, 12]],
[[-1, 1], [1, 2], [1, 3], [-5, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 7], [5, 9], [6, 7], [6, 8], [7, 10], [8, 9], [8, 11], [9, 12], [10, -10], [12, -12], [10, 11], [11, 12]],
[[-1, 1], [1, 2], [1, 3], [-5, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 7], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 10], [9, 12], [10, -10], [12, -12], [10, 11], [11, 12]],
[[-1, 1], [1, 2], [1, 3], [-5, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 7], [5, 9], [6, 7], [6, 8], [7, 10], [8, 9], [8, 12], [9, 11], [10, -10], [12, -12], [10, 11], [11, 12]],
[[-1, 1], [1, 2], [-3, 3], [1, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 7], [5, 9], [6, 7], [6, 8], [7, 10], [8, 9], [8, 11], [9, 12], [10, -10], [12, -12], [10, 11], [11, 12]],
[[-1, 1], [1, 2], [-3, 3], [1, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 7], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 10], [9, 12], [10, -10], [12, -12], [10, 11], [11, 12]],
[[-1, 1], [1, 2], [-3, 3], [1, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 7], [5, 9], [6, 7], [6, 8], [7, 10], [8, 9], [8, 12], [9, 11], [10, -10], [12, -12], [10, 11], [11, 12]],
[[-1, 1], [-2, 2], [1, 3], [1, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 7], [5, 9], [6, 7], [6, 8], [7, 10], [8, 9], [8, 11], [9, 12], [10, -10], [12, -12], [10, 11], [11, 12]],
[[-1, 1], [-2, 2], [1, 3], [1, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 7], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 10], [9, 12], [10, -10], [12, -12], [10, 11], [11, 12]],
[[-1, 1], [-2, 2], [1, 3], [1, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 7], [5, 9], [6, 7], [6, 8], [7, 10], [8, 9], [8, 12], [9, 11], [10, -10], [12, -12], [10, 11], [11, 12]],
[[-1, 1], [1, 2], [1, 3], [-5, 5], [2, 3], [2, 4], [3, 6], [4, 5], [4, 8], [5, 7], [6, 7], [6, 9], [7, 10], [8, 9], [8, 11], [9, 12], [10, -10], [12, -12], [10, 11], [11, 12]],
[[-1, 1], [1, 2], [1, 3], [-5, 5], [2, 3], [2, 4], [3, 6], [4, 5], [4, 8], [5, 7], [6, 7], [6, 9], [7, 11], [8, 9], [8, 10], [9, 12], [10, -10], [12, -12], [10, 11], [11, 12]],
[[-1, 1], [1, 2], [1, 3], [-5, 5], [2, 3], [2, 4], [3, 6], [4, 5], [4, 8], [5, 7], [6, 7], [6, 9], [7, 10], [8, 9], [8, 12], [9, 11], [10, -10], [12, -12], [10, 11], [11, 12]],
[[-1, 1], [1, 2], [-3, 3], [1, 5], [2, 3], [2, 4], [3, 6], [4, 5], [4, 8], [5, 7], [6, 7], [6, 9], [7, 10], [8, 9], [8, 11], [9, 12], [10, -10], [12, -12], [10, 11], [11, 12]],
[[-1, 1], [1, 2], [-3, 3], [1, 5], [2, 3], [2, 4], [3, 6], [4, 5], [4, 8], [5, 7], [6, 7], [6, 9], [7, 11], [8, 9], [8, 10], [9, 12], [10, -10], [12, -12], [10, 11], [11, 12]],
[[-1, 1], [1, 2], [-3, 3], [1, 5], [2, 3], [2, 4], [3, 6], [4, 5], [4, 8], [5, 7], [6, 7], [6, 9], [7, 10], [8, 9], [8, 12], [9, 11], [10, -10], [12, -12], [10, 11], [11, 12]],
[[-1, 1], [-2, 2], [1, 3], [1, 5], [2, 3], [2, 4], [3, 6], [4, 5], [4, 8], [5, 7], [6, 7], [6, 9], [7, 10], [8, 9], [8, 11], [9, 12], [10, -10], [12, -12], [10, 11], [11, 12]],
[[-1, 1], [-2, 2], [1, 3], [1, 5], [2, 3], [2, 4], [3, 6], [4, 5], [4, 8], [5, 7], [6, 7], [6, 9], [7, 11], [8, 9], [8, 10], [9, 12], [10, -10], [12, -12], [10, 11], [11, 12]],
[[-1, 1], [-2, 2], [1, 3], [1, 5], [2, 3], [2, 4], [3, 6], [4, 5], [4, 8], [5, 7], [6, 7], [6, 9], [7, 10], [8, 9], [8, 12], [9, 11], [10, -10], [12, -12], [10, 11], [11, 12]],
[[-1, 1], [1, 2], [1, 3], [-5, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 8], [5, 7], [6, 7], [6, 10], [7, 9], [8, 9], [8, 11], [9, 12], [10, -10], [12, -12], [10, 11], [11, 12]],
[[-1, 1], [1, 2], [1, 3], [-5, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 8], [5, 7], [6, 7], [6, 11], [7, 9], [8, 9], [8, 10], [9, 12], [10, -10], [12, -12], [10, 11], [11, 12]],
[[-1, 1], [1, 2], [1, 3], [-5, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 8], [5, 7], [6, 7], [6, 10], [7, 9], [8, 9], [8, 12], [9, 11], [10, -10], [12, -12], [10, 11], [11, 12]],
[[-1, 1], [1, 2], [-3, 3], [1, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 8], [5, 7], [6, 7], [6, 10], [7, 9], [8, 9], [8, 11], [9, 12], [10, -10], [12, -12], [10, 11], [11, 12]],
[[-1, 1], [1, 2], [-3, 3], [1, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 8], [5, 7], [6, 7], [6, 11], [7, 9], [8, 9], [8, 10], [9, 12], [10, -10], [12, -12], [10, 11], [11, 12]],
[[-1, 1], [1, 2], [-3, 3], [1, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 8], [5, 7], [6, 7], [6, 10], [7, 9], [8, 9], [8, 12], [9, 11], [10, -10], [12, -12], [10, 11], [11, 12]],
[[-1, 1], [-2, 2], [1, 3], [1, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 8], [5, 7], [6, 7], [6, 10], [7, 9], [8, 9], [8, 11], [9, 12], [10, -10], [12, -12], [10, 11], [11, 12]],
[[-1, 1], [-2, 2], [1, 3], [1, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 8], [5, 7], [6, 7], [6, 11], [7, 9], [8, 9], [8, 10], [9, 12], [10, -10], [12, -12], [10, 11], [11, 12]],
[[-1, 1], [-2, 2], [1, 3], [1, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 8], [5, 7], [6, 7], [6, 10], [7, 9], [8, 9], [8, 12], [9, 11], [10, -10], [12, -12], [10, 11], [11, 12]]]

In [ ]:
for g in G4:
    res = solve_lp_burl(g, 12)
    print(res.fun)
    if res.fun < 1/3:
        print(g)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.


1.0000000000109117
0.6666666667352017


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: OptimizeWarning: Solving system with option 'cholesky':True failed. It is normal for this to happen occasionally, especially as the solution is approached. However, if you see this frequently, consider setting option 'cholesky' to False.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: OptimizeWarning: Solving system with option 'sym_pos':True failed. It is normal for this to happen occasionally, especially as the solution is approached. However, if you see this frequently, consider setting option 'sym_pos' to False.
/usr/local/lib/python3.7/dist-packages/scipy/optimize/_linprog_ip.py:117: LinAlgWarning: Ill-conditioned matrix (rcond=3.58975e-19): result may not be accurate.
  return sp.linalg.solve(M, r, sym_pos=sym_pos)


0.6666666666689232
0.6666666670959257
0.3333333334278154
1.0000000001375395
0.3333333334278143
1.0000000002838612
0.6666666793551181
1.000000000016321
1.000000000003276
0.6666666714540871
0.6666666667087346


/usr/local/lib/python3.7/dist-packages/scipy/optimize/_linprog_ip.py:117: LinAlgWarning: Ill-conditioned matrix (rcond=4.00236e-19): result may not be accurate.
  return sp.linalg.solve(M, r, sym_pos=sym_pos)


...

0.6666666666680731
0.6666666667529122


/usr/local/lib/python3.7/dist-packages/scipy/optimize/_linprog_ip.py:117: LinAlgWarning: Ill-conditioned matrix (rcond=7.39287e-20): result may not be accurate.
  return sp.linalg.solve(M, r, sym_pos=sym_pos)


0.6666666666693667
0.66666667935512
1.0000000002838643
0.3333333334278132


/usr/local/lib/python3.7/dist-packages/scipy/optimize/_linprog_ip.py:117: LinAlgWarning: Ill-conditioned matrix (rcond=2.53594e-19): result may not be accurate.
  return sp.linalg.solve(M, r, sym_pos=sym_pos)


1.0000000000021243
0.6666666793551197
1.000000000137544


In [ ]:
G3 = [[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, 12], [11, -8], [12, -12]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, 12], [11, -8], [12, -12]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, 12], [11, -8], [12, -12]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, -7], [11, 12], [12, -12]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, -7], [11, 12], [12, -12]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, -7], [11, 12], [12, -12]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 10], [9, -6], [10, 11], [10, 12], [11, 12], [12, -12]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 10], [9, -6], [10, 11], [10, 12], [11, 12], [12, -12]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 10], [9, -6], [10, 11], [10, 12], [11, 12], [12, -12]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, 12], [11, -8], [12, -12]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, 12], [11, -8], [12, -12]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, 12], [11, -8], [12, -12]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, -7], [11, 12], [12, -12]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, -7], [11, 12], [12, -12]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, -7], [11, 12], [12, -12]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 10], [9, -6], [10, 11], [10, 12], [11, 12], [12, -12]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 10], [9, -6], [10, 11], [10, 12], [11, 12], [12, -12]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 10], [9, -6], [10, 11], [10, 12], [11, 12], [12, -12]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 6], [5, 8], [6, 7], [6, 10], [7, 9], [8, 9], [8, 11], [9, 12], [10, 11], [10, 12], [11, -8], [12, -12]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 6], [5, 8], [6, 7], [6, 10], [7, 9], [8, 9], [8, 11], [9, 12], [10, 11], [10, 12], [11, -8], [12, -12]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 6], [5, 8], [6, 7], [6, 10], [7, 9], [8, 9], [8, 11], [9, 12], [10, 11], [10, 12], [11, -8], [12, -12]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 6], [5, 8], [6, 7], [6, 10], [7, 9], [8, 9], [8, 11], [9, 12], [10, 11], [10, -7], [11, 12], [12, -12]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 6], [5, 8], [6, 7], [6, 10], [7, 9], [8, 9], [8, 11], [9, 12], [10, 11], [10, -7], [11, 12], [12, -12]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 6], [5, 8], [6, 7], [6, 10], [7, 9], [8, 9], [8, 11], [9, 12], [10, 11], [10, -7], [11, 12], [12, -12]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 6], [5, 8], [6, 7], [6, 10], [7, 9], [8, 9], [8, 11], [9, -6], [10, 11], [10, 12], [11, 12], [12, -12]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 6], [5, 8], [6, 7], [6, 10], [7, 9], [8, 9], [8, 11], [9, -6], [10, 11], [10, 12], [11, 12], [12, -12]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 6], [5, 8], [6, 7], [6, 10], [7, 9], [8, 9], [8, 11], [9, -6], [10, 11], [10, 12], [11, 12], [12, -12]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 10], [7, 9], [8, 9], [8, 12], [9, 11], [10, 11], [10, 12], [11, -8], [12, -12]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 10], [7, 9], [8, 9], [8, 12], [9, 11], [10, 11], [10, 12], [11, -8], [12, -12]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 10], [7, 9], [8, 9], [8, 12], [9, 11], [10, 11], [10, 12], [11, -8], [12, -12]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 10], [7, 9], [8, 9], [8, 12], [9, 11], [10, 11], [10, -7], [11, 12], [12, -12]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 10], [7, 9], [8, 9], [8, 12], [9, 11], [10, 11], [10, -7], [11, 12], [12, -12]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 10], [7, 9], [8, 9], [8, 12], [9, 11], [10, 11], [10, -7], [11, 12], [12, -12]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 10], [7, 9], [8, 9], [8, -5], [9, 11], [10, 11], [10, 12], [11, 12], [12, -12]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 10], [7, 9], [8, 9], [8, -5], [9, 11], [10, 11], [10, 12], [11, 12], [12, -12]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 10], [7, 9], [8, 9], [8, -5], [9, 11], [10, 11], [10, 12], [11, 12], [12, -12]]]

In [ ]:
for g in G4:
    res = solve_lp_burl(g, 12)
    print(res.fun)
    if res.fun < 1/3:
        print(g)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.


1.0000000000109117
0.6666666667352017


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: OptimizeWarning: Solving system with option 'cholesky':True failed. It is normal for this to happen occasionally, especially as the solution is approached. However, if you see this frequently, consider setting option 'cholesky' to False.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: OptimizeWarning: Solving system with option 'sym_pos':True failed. It is normal for this to happen occasionally, especially as the solution is approached. However, if you see this frequently, consider setting option 'sym_pos' to False.
/usr/local/lib/python3.7/dist-packages/scipy/optimize/_linprog_ip.py:117: LinAlgWarning: Ill-conditioned matrix (rcond=3.58975e-19): result may not be accurate.
  return sp.linalg.solve(M, r, sym_pos=sym_pos)


0.6666666666689232
0.6666666670959257
0.3333333334278154
1.0000000001375395
0.3333333334278143
1.0000000002838612
0.6666666793551181
1.000000000016321
1.000000000003276
0.6666666714540871
0.6666666667087346


/usr/local/lib/python3.7/dist-packages/scipy/optimize/_linprog_ip.py:117: LinAlgWarning: Ill-conditioned matrix (rcond=4.00236e-19): result may not be accurate.
  return sp.linalg.solve(M, r, sym_pos=sym_pos)


...

0.6666666666680731
0.6666666667529122


/usr/local/lib/python3.7/dist-packages/scipy/optimize/_linprog_ip.py:117: LinAlgWarning: Ill-conditioned matrix (rcond=7.39287e-20): result may not be accurate.
  return sp.linalg.solve(M, r, sym_pos=sym_pos)


0.6666666666693667
0.66666667935512
1.0000000002838643
0.3333333334278132


/usr/local/lib/python3.7/dist-packages/scipy/optimize/_linprog_ip.py:117: LinAlgWarning: Ill-conditioned matrix (rcond=2.53594e-19): result may not be accurate.
  return sp.linalg.solve(M, r, sym_pos=sym_pos)


1.0000000000021243
0.6666666793551197
1.000000000137544


In [ ]:
G3 = [[[-1, 1], [1, 2], [1, 3], [-5, 5], [2, 3], [2, 4], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 10], [8, 9], [8, 10], [9, -9], [10, -10]],
[[-1, 1], [1, 2], [1, 3], [-5, 5], [2, 3], [2, 4], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 10], [8, 9], [8, -8], [9, 10], [10, -10]],
[[-1, 1], [1, 2], [1, 3], [-5, 5], [2, 3], [2, 4], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, -7], [8, 9], [8, 10], [9, 10], [10, -10]],
[[-1, 1], [1, 2], [-3, 3], [1, 5], [2, 3], [2, 4], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 10], [8, 9], [8, 10], [9, -9], [10, -10]],
[[-1, 1], [1, 2], [-3, 3], [1, 5], [2, 3], [2, 4], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 10], [8, 9], [8, -8], [9, 10], [10, -10]],
[[-1, 1], [1, 2], [-3, 3], [1, 5], [2, 3], [2, 4], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, -7], [8, 9], [8, 10], [9, 10], [10, -10]],
[[-1, 1], [-2, 2], [1, 3], [1, 5], [2, 3], [2, 4], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 10], [8, 9], [8, 10], [9, -9], [10, -10]],
[[-1, 1], [-2, 2], [1, 3], [1, 5], [2, 3], [2, 4], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 10], [8, 9], [8, -8], [9, 10], [10, -10]],
[[-1, 1], [-2, 2], [1, 3], [1, 5], [2, 3], [2, 4], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, -7], [8, 9], [8, 10], [9, 10], [10, -10]],
[[-1, 1], [1, 2], [1, 3], [-5, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 7], [5, 9], [6, 7], [6, 8], [7, 10], [8, 9], [8, 10], [9, -9], [10, -10]],
[[-1, 1], [1, 2], [1, 3], [-5, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 7], [5, 9], [6, 7], [6, 8], [7, 10], [8, 9], [8, -8], [9, 10], [10, -10]],
[[-1, 1], [1, 2], [1, 3], [-5, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 7], [5, 9], [6, 7], [6, 8], [7, -7], [8, 9], [8, 10], [9, 10], [10, -10]],
[[-1, 1], [1, 2], [-3, 3], [1, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 7], [5, 9], [6, 7], [6, 8], [7, 10], [8, 9], [8, 10], [9, -9], [10, -10]],
[[-1, 1], [1, 2], [-3, 3], [1, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 7], [5, 9], [6, 7], [6, 8], [7, 10], [8, 9], [8, -8], [9, 10], [10, -10]],
[[-1, 1], [1, 2], [-3, 3], [1, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 7], [5, 9], [6, 7], [6, 8], [7, -7], [8, 9], [8, 10], [9, 10], [10, -10]],
[[-1, 1], [-2, 2], [1, 3], [1, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 7], [5, 9], [6, 7], [6, 8], [7, 10], [8, 9], [8, 10], [9, -9], [10, -10]],
[[-1, 1], [-2, 2], [1, 3], [1, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 7], [5, 9], [6, 7], [6, 8], [7, 10], [8, 9], [8, -8], [9, 10], [10, -10]],
[[-1, 1], [-2, 2], [1, 3], [1, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 7], [5, 9], [6, 7], [6, 8], [7, -7], [8, 9], [8, 10], [9, 10], [10, -10]],
[[-1, 1], [1, 2], [1, 3], [-5, 5], [2, 3], [2, 4], [3, 6], [4, 5], [4, 8], [5, 7], [6, 7], [6, 9], [7, 10], [8, 9], [8, 10], [9, -9], [10, -10]],
[[-1, 1], [1, 2], [1, 3], [-5, 5], [2, 3], [2, 4], [3, 6], [4, 5], [4, 8], [5, 7], [6, 7], [6, 9], [7, 10], [8, 9], [8, -8], [9, 10], [10, -10]],
[[-1, 1], [1, 2], [1, 3], [-5, 5], [2, 3], [2, 4], [3, 6], [4, 5], [4, 8], [5, 7], [6, 7], [6, 9], [7, -7], [8, 9], [8, 10], [9, 10], [10, -10]],
[[-1, 1], [1, 2], [-3, 3], [1, 5], [2, 3], [2, 4], [3, 6], [4, 5], [4, 8], [5, 7], [6, 7], [6, 9], [7, 10], [8, 9], [8, 10], [9, -9], [10, -10]],
[[-1, 1], [1, 2], [-3, 3], [1, 5], [2, 3], [2, 4], [3, 6], [4, 5], [4, 8], [5, 7], [6, 7], [6, 9], [7, 10], [8, 9], [8, -8], [9, 10], [10, -10]],
[[-1, 1], [1, 2], [-3, 3], [1, 5], [2, 3], [2, 4], [3, 6], [4, 5], [4, 8], [5, 7], [6, 7], [6, 9], [7, -7], [8, 9], [8, 10], [9, 10], [10, -10]],
[[-1, 1], [-2, 2], [1, 3], [1, 5], [2, 3], [2, 4], [3, 6], [4, 5], [4, 8], [5, 7], [6, 7], [6, 9], [7, 10], [8, 9], [8, 10], [9, -9], [10, -10]],
[[-1, 1], [-2, 2], [1, 3], [1, 5], [2, 3], [2, 4], [3, 6], [4, 5], [4, 8], [5, 7], [6, 7], [6, 9], [7, 10], [8, 9], [8, -8], [9, 10], [10, -10]],
[[-1, 1], [-2, 2], [1, 3], [1, 5], [2, 3], [2, 4], [3, 6], [4, 5], [4, 8], [5, 7], [6, 7], [6, 9], [7, -7], [8, 9], [8, 10], [9, 10], [10, -10]],
[[-1, 1], [1, 2], [1, 3], [-5, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 8], [5, 7], [6, 7], [6, 10], [7, 9], [8, 9], [8, 10], [9, -9], [10, -10]],
[[-1, 1], [1, 2], [1, 3], [-5, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 8], [5, 7], [6, 7], [6, 10], [7, 9], [8, 9], [8, -8], [9, 10], [10, -10]],
[[-1, 1], [1, 2], [1, 3], [-5, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 8], [5, 7], [6, 7], [6, -6], [7, 9], [8, 9], [8, 10], [9, 10], [10, -10]],
[[-1, 1], [1, 2], [-3, 3], [1, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 8], [5, 7], [6, 7], [6, 10], [7, 9], [8, 9], [8, 10], [9, -9], [10, -10]],
[[-1, 1], [1, 2], [-3, 3], [1, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 8], [5, 7], [6, 7], [6, 10], [7, 9], [8, 9], [8, -8], [9, 10], [10, -10]],
[[-1, 1], [1, 2], [-3, 3], [1, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 8], [5, 7], [6, 7], [6, -6], [7, 9], [8, 9], [8, 10], [9, 10], [10, -10]],
[[-1, 1], [-2, 2], [1, 3], [1, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 8], [5, 7], [6, 7], [6, 10], [7, 9], [8, 9], [8, 10], [9, -9], [10, -10]],
[[-1, 1], [-2, 2], [1, 3], [1, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 8], [5, 7], [6, 7], [6, 10], [7, 9], [8, 9], [8, -8], [9, 10], [10, -10]],
[[-1, 1], [-2, 2], [1, 3], [1, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 8], [5, 7], [6, 7], [6, -6], [7, 9], [8, 9], [8, 10], [9, 10], [10, -10]]]

In [ ]:
for g in G3:
    res = solve_lp_burl(g, 10)
    print(res.fun)
    if res.fun < 1/3:
        print(g)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: OptimizeWarning: Solving system with option 'cholesky':True failed. It is normal for this to happen occasionally, especially as the solution is approached. However, if you see this frequently, consider setting option 'cholesky' to False.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: OptimizeWarning: Solving system with option 'sym_pos':True failed. It is normal for this to happen occasionally, especially as the solution is approached. However, if you see this frequently, consider setting option 'sym_pos' to False.
/usr/local/lib/python3.7/dist-packages/scipy/optimize/_linprog_ip.py:117: LinAlgWarning: Ill-conditioned matrix (rcond=4.60185e-19): result may not be accurate.
  return sp.linalg

0.33333333333444437
1.0000000001989502
0.6666666666733224
1.000000000002149
0.6666666666852933
0.33333333333439824
0.6666666666852928


/usr/local/lib/python3.7/dist-packages/scipy/optimize/_linprog_ip.py:117: LinAlgWarning: Ill-conditioned matrix (rcond=2.89391e-19): result may not be accurate.
  return sp.linalg.solve(M, r, sym_pos=sym_pos)


0.33333333333381454
1.000000000199011
0.6666666666752403
1.0000000001989726
0.6666666718065763
0.6666666666797216


/usr/local/lib/python3.7/dist-packages/scipy/optimize/_linprog_ip.py:117: LinAlgWarning: Ill-conditioned matrix (rcond=3.95974e-20): result may not be accurate.
  return sp.linalg.solve(M, r, sym_pos=sym_pos)


...

0.6666666666679326
0.6666666772093934
0.3333333333346389
0.6666666666797356
0.6666666697710752
0.6666666772093945


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: OptimizeWarning: Solving system with option 'sym_pos':False failed. This may happen occasionally, especially as the solution is approached. However, if you see this frequently, your problem may be numerically challenging. If you cannot improve the formulation, consider setting 'lstsq' to True. Consider also setting `presolve` to True, if it is not already.


0.33333333333381054
0.6666666666797163
0.6666666666679372
0.6666666666679366
0.6666666666861574
0.6666666718065807


/usr/local/lib/python3.7/dist-packages/scipy/optimize/_linprog_ip.py:117: LinAlgWarning: Ill-conditioned matrix (rcond=1.55845e-19): result may not be accurate.
  return sp.linalg.solve(M, r, sym_pos=sym_pos)


0.3333333333338139
0.6666666666852936
1.000000000198979
0.6666666666797234
0.6666666737479621
0.6666666666752398


In [ ]:
G2 = [[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, 13], [11, 14], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 10], [9, 13], [10, 11], [10, 12], [11, 14], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, 14], [11, 13], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, 13], [11, 14], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 10], [9, 13], [10, 11], [10, 12], [11, 14], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, 14], [11, 13], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, 13], [11, 14], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 10], [9, 13], [10, 11], [10, 12], [11, 14], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, 14], [11, 13], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, 13], [11, 14], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 10], [9, 13], [10, 11], [10, 12], [11, 14], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, 14], [11, 13], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, 13], [11, 14], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 10], [9, 13], [10, 11], [10, 12], [11, 14], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, 14], [11, 13], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, 13], [11, 14], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 10], [9, 13], [10, 11], [10, 12], [11, 14], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, 14], [11, 13], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 6], [5, 8], [6, 7], [6, 10], [7, 9], [8, 9], [8, 11], [9, 12], [10, 11], [10, 13], [11, 14], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 6], [5, 8], [6, 7], [6, 10], [7, 9], [8, 9], [8, 11], [9, 13], [10, 11], [10, 12], [11, 14], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 6], [5, 8], [6, 7], [6, 10], [7, 9], [8, 9], [8, 11], [9, 12], [10, 11], [10, 14], [11, 13], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 6], [5, 8], [6, 7], [6, 10], [7, 9], [8, 9], [8, 11], [9, 12], [10, 11], [10, 13], [11, 14], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 6], [5, 8], [6, 7], [6, 10], [7, 9], [8, 9], [8, 11], [9, 13], [10, 11], [10, 12], [11, 14], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 6], [5, 8], [6, 7], [6, 10], [7, 9], [8, 9], [8, 11], [9, 12], [10, 11], [10, 14], [11, 13], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 6], [5, 8], [6, 7], [6, 10], [7, 9], [8, 9], [8, 11], [9, 12], [10, 11], [10, 13], [11, 14], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 6], [5, 8], [6, 7], [6, 10], [7, 9], [8, 9], [8, 11], [9, 13], [10, 11], [10, 12], [11, 14], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 6], [5, 8], [6, 7], [6, 10], [7, 9], [8, 9], [8, 11], [9, 12], [10, 11], [10, 14], [11, 13], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 10], [7, 9], [8, 9], [8, 12], [9, 11], [10, 11], [10, 13], [11, 14], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 10], [7, 9], [8, 9], [8, 13], [9, 11], [10, 11], [10, 12], [11, 14], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 10], [7, 9], [8, 9], [8, 12], [9, 11], [10, 11], [10, 14], [11, 13], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 10], [7, 9], [8, 9], [8, 12], [9, 11], [10, 11], [10, 13], [11, 14], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 10], [7, 9], [8, 9], [8, 13], [9, 11], [10, 11], [10, 12], [11, 14], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 10], [7, 9], [8, 9], [8, 12], [9, 11], [10, 11], [10, 14], [11, 13], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 10], [7, 9], [8, 9], [8, 12], [9, 11], [10, 11], [10, 13], [11, 14], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 10], [7, 9], [8, 9], [8, 13], [9, 11], [10, 11], [10, 12], [11, 14], [12, -12], [14, -14], [12, 13], [13, 14]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 10], [7, 9], [8, 9], [8, 12], [9, 11], [10, 11], [10, 14], [11, 13], [12, -12], [14, -14], [12, 13], [13, 14]]]

In [ ]:
for g in G2:
    res = solve_lp_burl(g, 14)
    print(res.fun)
    if res.fun < 1/3:
        print(g)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.


1.0000000000057276
0.6666666666815374
1.333333333495523
0.6666666763924634
1.0000000023784008
1.000000000005771
1.0000000023784037
0.6666666667498655
0.6666666666815484
0.6666666666757783
0.6666666718657938
1.0000000002170266
1.0000000000378273
0.666666666675726
1.333333333596693
1.0000000005198237
0.8333333333343844
0.6666666666893136
1.3333333333867718
0.6666666667394753
1.0000000213937954
1.0000000213937978
0.6666666770524583
1.0000000000072193
0.6666666667394919
1.0000000000522797
0.6666666770524582
1.00000000003783
1.0000000005198257
0.6666666666757883
1.333333333596684
0.6666666666893128
1.0000000002170168
0.666666666675736


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: OptimizeWarning: Solving system with option 'cholesky':True failed. It is normal for this to happen occasionally, especially as the solution is approached. However, if you see this frequently, consider setting option 'cholesky' to False.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: OptimizeWarning: Solving system with option 'sym_pos':True failed. It is normal for this to happen occasionally, especially as the solution is approached. However, if you see this frequently, consider setting option 'sym_pos' to False.
/usr/local/lib/python3.7/dist-packages/scipy/optimize/_linprog_ip.py:117: LinAlgWarning: Ill-conditioned matrix (rcond=1.35345e-18): result may not be accurate.
  return sp.linalg.solve(M, r, sym_pos=sym_pos)


0.8333333333343845
0.6666666718657936


In [ ]:
Gb3 = [[[-1, 1], [1, 2], [1, 3], [-5, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 7], [5, 9], [6, 7], [6, 10], [7, 8], [8, 9], [8, 11], [9, 13], [10, 11], [10, -10], [11, 12], [12, 13], [12, -12], [13, -13]],
[[-1, 1], [-2, 2], [-4, 4], [1, 2], [1, 5], [2, 3], [3, 4], [3, 6], [4, 8], [5, 6], [5, 9], [6, 7], [7, 8], [7, 10], [8, 12], [9, 10], [9, 13], [10, 11], [11, 12], [11, 13], [12, -12], [13, -13]],
[[-1, 1], [1, 2], [-3, 3], [1, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 7], [5, 9], [6, 7], [6, 10], [7, 8], [8, 9], [8, 11], [9, 13], [10, 11], [10, -10], [11, 12], [12, 13], [12, -12], [13, -13]],
[[-1, 1], [-2, 2], [-4, 4], [1, 2], [1, 5], [2, 3], [3, 4], [3, 6], [4, 8], [5, 6], [5, 9], [6, 7], [7, 8], [7, 10], [8, 12], [9, 10], [9, 13], [10, 11], [11, 12], [11, -11], [12, 13], [13, -13]],
[[-1, 1], [-2, 2], [1, 3], [1, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 7], [5, 9], [6, 7], [6, 10], [7, 8], [8, 9], [8, 11], [9, 13], [10, 11], [10, -10], [11, 12], [12, 13], [12, -12], [13, -13]],
[[-1, 1], [-2, 2], [-4, 4], [1, 2], [1, 5], [2, 3], [3, 4], [3, 6], [4, 8], [5, 6], [5, 9], [6, 7], [7, 8], [7, 10], [8, 12], [9, 10], [9, -9], [10, 11], [11, 12], [11, 13], [12, 13], [13, -13]],
[[-1, 1], [1, 2], [1, 3], [-5, 5], [2, 3], [2, 4], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 10], [8, 9], [8, 12], [9, 11], [10, 11], [10, 13], [11, -11], [12, 13], [12, -12], [13, -13]],
[[-1, 1], [-2, 2], [-4, 4], [1, 2], [1, 3], [2, 6], [3, 4], [3, 5], [4, 8], [5, 6], [5, 7], [6, 9], [7, 8], [7, 11], [8, 10], [9, 10], [9, 12], [10, 13], [11, 12], [11, 13], [12, -12], [13, -13]],
[[-1, 1], [1, 2], [-3, 3], [1, 5], [2, 3], [2, 4], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 10], [8, 9], [8, 12], [9, 11], [10, 11], [10, 13], [11, -11], [12, 13], [12, -12], [13, -13]],
[[-1, 1], [-2, 2], [-4, 4], [1, 2], [1, 3], [2, 6], [3, 4], [3, 5], [4, 8], [5, 6], [5, 7], [6, 9], [7, 8], [7, 11], [8, 10], [9, 10], [9, 12], [10, 13], [11, 12], [11, -11], [12, 13], [13, -13]],
[[-1, 1], [-2, 2], [1, 3], [1, 5], [2, 3], [2, 4], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 10], [8, 9], [8, 12], [9, 11], [10, 11], [10, 13], [11, -11], [12, 13], [12, -12], [13, -13]],
[[-1, 1], [-2, 2], [-4, 4], [1, 2], [1, 3], [2, 6], [3, 4], [3, 5], [4, 8], [5, 6], [5, 7], [6, 9], [7, 8], [7, 11], [8, 10], [9, 10], [9, 12], [10, -10], [11, 12], [11, 13], [12, 13], [13, -13]],
[[-1, 1], [1, 2], [1, 3], [-5, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 7], [5, 9], [6, 7], [6, 10], [7, 8], [8, 9], [8, 12], [9, 11], [10, 11], [10, -10], [11, 13], [12, 13], [12, -12], [13, -13]],
[[-1, 1], [-2, 2], [-4, 4], [1, 2], [1, 5], [2, 3], [3, 4], [3, 6], [4, 8], [5, 6], [5, 9], [6, 7], [7, 8], [7, 11], [8, 10], [9, 10], [9, 13], [10, 12], [11, 12], [11, 13], [12, -12], [13, -13]],
[[-1, 1], [1, 2], [-3, 3], [1, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 7], [5, 9], [6, 7], [6, 10], [7, 8], [8, 9], [8, 12], [9, 11], [10, 11], [10, -10], [11, 13], [12, 13], [12, -12], [13, -13]],
[[-1, 1], [-2, 2], [-4, 4], [1, 2], [1, 5], [2, 3], [3, 4], [3, 6], [4, 8], [5, 6], [5, 9], [6, 7], [7, 8], [7, 11], [8, 10], [9, 10], [9, 13], [10, 12], [11, 12], [11, -11], [12, 13], [13, -13]],
[[-1, 1], [-2, 2], [1, 3], [1, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 7], [5, 9], [6, 7], [6, 10], [7, 8], [8, 9], [8, 12], [9, 11], [10, 11], [10, -10], [11, 13], [12, 13], [12, -12], [13, -13]],
[[-1, 1], [-2, 2], [-4, 4], [1, 2], [1, 5], [2, 3], [3, 4], [3, 6], [4, 8], [5, 6], [5, 9], [6, 7], [7, 8], [7, 11], [8, 10], [9, 10], [9, -9], [10, 12], [11, 12], [11, 13], [12, 13], [13, -13]],
[[-1, 1], [1, 2], [1, 3], [-5, 5], [2, 3], [2, 4], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 10], [7, 8], [8, 9], [8, 11], [9, 13], [10, 11], [10, -10], [11, 12], [12, 13], [12, -12], [13, -13]],
[[-1, 1], [-2, 2], [-4, 4], [1, 2], [1, 3], [2, 6], [3, 4], [3, 5], [4, 8], [5, 6], [5, 9], [6, 7], [7, 8], [7, 10], [8, 12], [9, 10], [9, 13], [10, 11], [11, 12], [11, 13], [12, -12], [13, -13]],
[[-1, 1], [1, 2], [-3, 3], [1, 5], [2, 3], [2, 4], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 10], [7, 8], [8, 9], [8, 11], [9, 13], [10, 11], [10, -10], [11, 12], [12, 13], [12, -12], [13, -13]],
[[-1, 1], [-2, 2], [-4, 4], [1, 2], [1, 3], [2, 6], [3, 4], [3, 5], [4, 8], [5, 6], [5, 9], [6, 7], [7, 8], [7, 10], [8, 12], [9, 10], [9, 13], [10, 11], [11, 12], [11, -11], [12, 13], [13, -13]],
[[-1, 1], [-2, 2], [1, 3], [1, 5], [2, 3], [2, 4], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 10], [7, 8], [8, 9], [8, 11], [9, 13], [10, 11], [10, -10], [11, 12], [12, 13], [12, -12], [13, -13]],
[[-1, 1], [-2, 2], [-4, 4], [1, 2], [1, 3], [2, 6], [3, 4], [3, 5], [4, 8], [5, 6], [5, 9], [6, 7], [7, 8], [7, 10], [8, 12], [9, 10], [9, -9], [10, 11], [11, 12], [11, 13], [12, 13], [13, -13]],
[[-1, 1], [1, 2], [1, 3], [-5, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 7], [5, 9], [6, 7], [6, 8], [7, 10], [8, 9], [8, 12], [9, 11], [10, 11], [10, -10], [11, 13], [12, 13], [12, -12], [13, -13]],
[[-1, 1], [-2, 2], [-4, 4], [1, 2], [1, 5], [2, 3], [3, 4], [3, 6], [4, 8], [5, 6], [5, 7], [6, 9], [7, 8], [7, 11], [8, 10], [9, 10], [9, 13], [10, 12], [11, 12], [11, 13], [12, -12], [13, -13]],
[[-1, 1], [1, 2], [-3, 3], [1, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 7], [5, 9], [6, 7], [6, 8], [7, 10], [8, 9], [8, 12], [9, 11], [10, 11], [10, -10], [11, 13], [12, 13], [12, -12], [13, -13]],
[[-1, 1], [-2, 2], [-4, 4], [1, 2], [1, 5], [2, 3], [3, 4], [3, 6], [4, 8], [5, 6], [5, 7], [6, 9], [7, 8], [7, 11], [8, 10], [9, 10], [9, 13], [10, 12], [11, 12], [11, -11], [12, 13], [13, -13]],
[[-1, 1], [-2, 2], [1, 3], [1, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 7], [5, 9], [6, 7], [6, 8], [7, 10], [8, 9], [8, 12], [9, 11], [10, 11], [10, -10], [11, 13], [12, 13], [12, -12], [13, -13]],
[[-1, 1], [-2, 2], [-4, 4], [1, 2], [1, 5], [2, 3], [3, 4], [3, 6], [4, 8], [5, 6], [5, 7], [6, 9], [7, 8], [7, 11], [8, 10], [9, 10], [9, -9], [10, 12], [11, 12], [11, 13], [12, 13], [13, -13]],
[[-1, 1], [1, 2], [1, 3], [-5, 5], [2, 3], [2, 4], [3, 6], [4, 5], [4, 8], [5, 7], [6, 7], [6, 9], [7, 11], [8, 9], [8, 12], [9, 10], [10, 11], [10, 13], [11, -11], [12, 13], [12, -12], [13, -13]],
[[-1, 1], [-2, 2], [-4, 4], [1, 2], [1, 3], [2, 5], [3, 4], [3, 7], [4, 6], [5, 6], [5, 8], [6, 10], [7, 8], [7, 11], [8, 9], [9, 10], [9, 12], [10, 13], [11, 12], [11, 13], [12, -12], [13, -13]],
[[-1, 1], [1, 2], [-3, 3], [1, 5], [2, 3], [2, 4], [3, 6], [4, 5], [4, 8], [5, 7], [6, 7], [6, 9], [7, 11], [8, 9], [8, 12], [9, 10], [10, 11], [10, 13], [11, -11], [12, 13], [12, -12], [13, -13]],
[[-1, 1], [-2, 2], [-4, 4], [1, 2], [1, 3], [2, 5], [3, 4], [3, 7], [4, 6], [5, 6], [5, 8], [6, 10], [7, 8], [7, 11], [8, 9], [9, 10], [9, 12], [10, 13], [11, 12], [11, -11], [12, 13], [13, -13]],
[[-1, 1], [-2, 2], [1, 3], [1, 5], [2, 3], [2, 4], [3, 6], [4, 5], [4, 8], [5, 7], [6, 7], [6, 9], [7, 11], [8, 9], [8, 12], [9, 10], [10, 11], [10, 13], [11, -11], [12, 13], [12, -12], [13, -13]],
[[-1, 1], [-2, 2], [-4, 4], [1, 2], [1, 3], [2, 5], [3, 4], [3, 7], [4, 6], [5, 6], [5, 8], [6, 10], [7, 8], [7, 11], [8, 9], [9, 10], [9, 12], [10, -10], [11, 12], [11, 13], [12, 13], [13, -13]],
[[-1, 1], [1, 2], [1, 3], [-5, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 9], [5, 7], [6, 7], [6, 8], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, -10], [11, 13], [12, 13], [12, -12], [13, -13]],
[[-1, 1], [-2, 2], [-4, 4], [1, 2], [1, 5], [2, 3], [3, 4], [3, 8], [4, 6], [5, 6], [5, 7], [6, 10], [7, 8], [7, 9], [8, 11], [9, 10], [9, 13], [10, 12], [11, 12], [11, 13], [12, -12], [13, -13]],
[[-1, 1], [1, 2], [-3, 3], [1, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 9], [5, 7], [6, 7], [6, 8], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, -10], [11, 13], [12, 13], [12, -12], [13, -13]],
[[-1, 1], [-2, 2], [-4, 4], [1, 2], [1, 5], [2, 3], [3, 4], [3, 8], [4, 6], [5, 6], [5, 7], [6, 10], [7, 8], [7, 9], [8, 11], [9, 10], [9, 13], [10, 12], [11, 12], [11, -11], [12, 13], [13, -13]],
[[-1, 1], [-2, 2], [1, 3], [1, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 9], [5, 7], [6, 7], [6, 8], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, -10], [11, 13], [12, 13], [12, -12], [13, -13]],
[[-1, 1], [-2, 2], [-4, 4], [1, 2], [1, 5], [2, 3], [3, 4], [3, 8], [4, 6], [5, 6], [5, 7], [6, 10], [7, 8], [7, 9], [8, 11], [9, 10], [9, -9], [10, 12], [11, 12], [11, 13], [12, 13], [13, -13]],
[[-1, 1], [1, 2], [1, 3], [-5, 5], [2, 3], [2, 4], [3, 6], [4, 5], [4, 8], [5, 7], [6, 7], [6, 11], [7, 9], [8, 9], [8, 10], [9, 13], [10, 11], [10, -10], [11, 12], [12, 13], [12, -12], [13, -13]],
[[-1, 1], [-2, 2], [-4, 4], [1, 2], [1, 3], [2, 5], [3, 4], [3, 7], [4, 6], [5, 6], [5, 10], [6, 8], [7, 8], [7, 9], [8, 12], [9, 10], [9, 13], [10, 11], [11, 12], [11, 13], [12, -12], [13, -13]],
[[-1, 1], [1, 2], [-3, 3], [1, 5], [2, 3], [2, 4], [3, 6], [4, 5], [4, 8], [5, 7], [6, 7], [6, 11], [7, 9], [8, 9], [8, 10], [9, 13], [10, 11], [10, -10], [11, 12], [12, 13], [12, -12], [13, -13]],
[[-1, 1], [-2, 2], [-4, 4], [1, 2], [1, 3], [2, 5], [3, 4], [3, 7], [4, 6], [5, 6], [5, 10], [6, 8], [7, 8], [7, 9], [8, 12], [9, 10], [9, 13], [10, 11], [11, 12], [11, -11], [12, 13], [13, -13]],
[[-1, 1], [-2, 2], [1, 3], [1, 5], [2, 3], [2, 4], [3, 6], [4, 5], [4, 8], [5, 7], [6, 7], [6, 11], [7, 9], [8, 9], [8, 10], [9, 13], [10, 11], [10, -10], [11, 12], [12, 13], [12, -12], [13, -13]],
[[-1, 1], [-2, 2], [-4, 4], [1, 2], [1, 3], [2, 5], [3, 4], [3, 7], [4, 6], [5, 6], [5, 10], [6, 8], [7, 8], [7, 9], [8, 12], [9, 10], [9, -9], [10, 11], [11, 12], [11, 13], [12, 13], [13, -13]],
[[-1, 1], [1, 2], [1, 3], [-5, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 8], [5, 7], [6, 7], [6, 10], [7, 9], [8, 9], [8, 13], [9, 11], [10, 11], [10, 12], [11, -11], [12, 13], [12, -12], [13, -13]],
[[-1, 1], [-2, 2], [-4, 4], [1, 2], [1, 5], [2, 3], [3, 4], [3, 7], [4, 6], [5, 6], [5, 9], [6, 8], [7, 8], [7, 12], [8, 10], [9, 10], [9, 11], [10, 13], [11, 12], [11, 13], [12, -12], [13, -13]],
[[-1, 1], [1, 2], [-3, 3], [1, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 8], [5, 7], [6, 7], [6, 10], [7, 9], [8, 9], [8, 13], [9, 11], [10, 11], [10, 12], [11, -11], [12, 13], [12, -12], [13, -13]],
[[-1, 1], [-2, 2], [-4, 4], [1, 2], [1, 5], [2, 3], [3, 4], [3, 7], [4, 6], [5, 6], [5, 9], [6, 8], [7, 8], [7, 12], [8, 10], [9, 10], [9, 11], [10, 13], [11, 12], [11, -11], [12, 13], [13, -13]],
[[-1, 1], [-2, 2], [1, 3], [1, 5], [2, 3], [2, 6], [3, 4], [4, 5], [4, 8], [5, 7], [6, 7], [6, 10], [7, 9], [8, 9], [8, 13], [9, 11], [10, 11], [10, 12], [11, -11], [12, 13], [12, -12], [13, -13]],
[[-1, 1], [-2, 2], [-4, 4], [1, 2], [1, 5], [2, 3], [3, 4], [3, 7], [4, 6], [5, 6], [5, 9], [6, 8], [7, 8], [7, 12], [8, 10], [9, 10], [9, 11], [10, -10], [11, 12], [11, 13], [12, 13], [13, -13]]]
for g in Gb3:
    res = solve_lp_burl(g, 13)
    print(res.fun)
    if res.fun < 1/3:
        print(g)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.


0.3333333334903854
0.33333333334379517
0.666666672033656
0.6666666720336591
0.33333333334377235
0.3333333334903875
0.3333333333451436
0.6666666736447535
0.6666666667129604
0.6666666668787036
0.33333333334227394
0.3333333382387901
0.33333333333754434
0.6666666736447486
0.6666666816239519
0.6666666668434229


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: OptimizeWarning: Solving system with option 'cholesky':True failed. It is normal for this to happen occasionally, especially as the solution is approached. However, if you see this frequently, consider setting option 'cholesky' to False.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: OptimizeWarning: Solving system with option 'sym_pos':True failed. It is normal for this to happen occasionally, especially as the solution is approached. However, if you see this frequently, consider setting option 'sym_pos' to False.
/usr/local/lib/python3.7/dist-packages/scipy/optimize/_linprog_ip.py:117: LinAlgWarning: Ill-conditioned matrix (rcond=2.42145e-18): result may not be accurate.
  return sp.linalg.solve(M, r, sym_pos=sym_pos)


0.33333333333833287
0.666666666667568


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: OptimizeWarning: Solving system with option 'sym_pos':False failed. This may happen occasionally, especially as the solution is approached. However, if you see this frequently, your problem may be numerically challenging. If you cannot improve the formulation, consider setting 'lstsq' to True. Consider also setting `presolve` to True, if it is not already.


...

/usr/local/lib/python3.7/dist-packages/scipy/optimize/_linprog_ip.py:117: LinAlgWarning: Ill-conditioned matrix (rcond=5.25177e-19): result may not be accurate.
  return sp.linalg.solve(M, r, sym_pos=sym_pos)


0.3333333333347833
0.3333333333347805
0.6666666667145524
0.6666666667403772
0.33333333344918725
0.3333333333422749
0.33333333333934373


/usr/local/lib/python3.7/dist-packages/scipy/optimize/_linprog_ip.py:117: LinAlgWarning: Ill-conditioned matrix (rcond=1.95401e-19): result may not be accurate.
  return sp.linalg.solve(M, r, sym_pos=sym_pos)


0.3333333333345454
0.6666666666831913
0.3333333333457653
0.3333333333457621
0.6666666666831915
0.3333333333345416


/usr/local/lib/python3.7/dist-packages/scipy/optimize/_linprog_ip.py:117: LinAlgWarning: Ill-conditioned matrix (rcond=3.33328e-19): result may not be accurate.
  return sp.linalg.solve(M, r, sym_pos=sym_pos)


In [ ]:
Gk4 = [[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 10], [9, 13], [10, 11], [10, 12], [11, -11], [12, 13], [12, -12], [13, -13]],
[[-1, 1], [-2, 2], [-4, 4], [1, 2], [1, 3], [2, 6], [3, 4], [3, 5], [4, 8], [5, 6], [5, 7], [6, 10], [7, 8], [7, 9], [8, 11], [9, 10], [9, 12], [10, 13], [11, -11], [13, -13], [11, 12], [12, 13]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 10], [9, 13], [10, 11], [10, 12], [11, -11], [12, 13], [12, -12], [13, -13]],
[[-1, 1], [-2, 2], [-4, 4], [1, 2], [1, 3], [2, 6], [3, 4], [3, 5], [4, 8], [5, 6], [5, 7], [6, 10], [7, 8], [7, 9], [8, 12], [9, 10], [9, 11], [10, 13], [11, -11], [13, -13], [11, 12], [12, 13]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 10], [9, 13], [10, 11], [10, 12], [11, -11], [12, 13], [12, -12], [13, -13]],
[[-1, 1], [-2, 2], [-4, 4], [1, 2], [1, 3], [2, 6], [3, 4], [3, 5], [4, 8], [5, 6], [5, 7], [6, 10], [7, 8], [7, 9], [8, 11], [9, 10], [9, 13], [10, 12], [11, -11], [13, -13], [11, 12], [12, 13]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 12], [9, 10], [10, 11], [10, -10], [11, 13], [12, 13], [12, -12], [13, -13]],
[[-1, 1], [-2, 2], [-4, 4], [1, 2], [1, 5], [2, 3], [3, 4], [3, 6], [4, 8], [5, 6], [5, 9], [6, 7], [7, 8], [7, 11], [8, 10], [9, 10], [9, 12], [10, 13], [11, -11], [13, -13], [11, 12], [12, 13]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 12], [9, 10], [10, 11], [10, -10], [11, 13], [12, 13], [12, -12], [13, -13]],
[[-1, 1], [-2, 2], [-4, 4], [1, 2], [1, 5], [2, 3], [3, 4], [3, 6], [4, 8], [5, 6], [5, 9], [6, 7], [7, 8], [7, 12], [8, 10], [9, 10], [9, 11], [10, 13], [11, -11], [13, -13], [11, 12], [12, 13]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 12], [9, 10], [10, 11], [10, -10], [11, 13], [12, 13], [12, -12], [13, -13]],
[[-1, 1], [-2, 2], [-4, 4], [1, 2], [1, 5], [2, 3], [3, 4], [3, 6], [4, 8], [5, 6], [5, 9], [6, 7], [7, 8], [7, 11], [8, 10], [9, 10], [9, 13], [10, 12], [11, -11], [13, -13], [11, 12], [12, 13]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 12], [9, 10], [10, 11], [10, 13], [11, -11], [12, 13], [12, -12], [13, -13]],
[[-1, 1], [-2, 2], [-4, 4], [1, 2], [1, 3], [2, 6], [3, 4], [3, 5], [4, 8], [5, 6], [5, 9], [6, 7], [7, 8], [7, 10], [8, 11], [9, 10], [9, 12], [10, 13], [11, -11], [13, -13], [11, 12], [12, 13]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 12], [9, 10], [10, 11], [10, 13], [11, -11], [12, 13], [12, -12], [13, -13]],
[[-1, 1], [-2, 2], [-4, 4], [1, 2], [1, 3], [2, 6], [3, 4], [3, 5], [4, 8], [5, 6], [5, 9], [6, 7], [7, 8], [7, 10], [8, 12], [9, 10], [9, 11], [10, 13], [11, -11], [13, -13], [11, 12], [12, 13]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 6], [5, 9], [6, 7], [6, 8], [7, 11], [8, 9], [8, 12], [9, 10], [10, 11], [10, 13], [11, -11], [12, 13], [12, -12], [13, -13]],
[[-1, 1], [-2, 2], [-4, 4], [1, 2], [1, 3], [2, 6], [3, 4], [3, 5], [4, 8], [5, 6], [5, 9], [6, 7], [7, 8], [7, 10], [8, 11], [9, 10], [9, 13], [10, 12], [11, -11], [13, -13], [11, 12], [12, 13]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, -10], [11, 13], [12, 13], [12, -12], [13, -13]],
[[-1, 1], [-2, 2], [-4, 4], [1, 2], [1, 5], [2, 3], [3, 4], [3, 6], [4, 8], [5, 6], [5, 7], [6, 9], [7, 8], [7, 11], [8, 10], [9, 10], [9, 12], [10, 13], [11, -11], [13, -13], [11, 12], [12, 13]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, -10], [11, 13], [12, 13], [12, -12], [13, -13]],
[[-1, 1], [-2, 2], [-4, 4], [1, 2], [1, 5], [2, 3], [3, 4], [3, 6], [4, 8], [5, 6], [5, 7], [6, 9], [7, 8], [7, 12], [8, 10], [9, 10], [9, 11], [10, 13], [11, -11], [13, -13], [11, 12], [12, 13]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 9], [7, 11], [8, 9], [8, 10], [9, 12], [10, 11], [10, -10], [11, 13], [12, 13], [12, -12], [13, -13]],
[[-1, 1], [-2, 2], [-4, 4], [1, 2], [1, 5], [2, 3], [3, 4], [3, 6], [4, 8], [5, 6], [5, 7], [6, 9], [7, 8], [7, 11], [8, 10], [9, 10], [9, 13], [10, 12], [11, -11], [13, -13], [11, 12], [12, 13]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 6], [5, 8], [6, 7], [6, 10], [7, 9], [8, 9], [8, 11], [9, 13], [10, 11], [10, -10], [11, 12], [12, 13], [12, -12], [13, -13]],
[[-1, 1], [-2, 2], [-4, 4], [1, 2], [1, 3], [2, 5], [3, 4], [3, 7], [4, 6], [5, 6], [5, 8], [6, 10], [7, 8], [7, 11], [8, 9], [9, 10], [9, 12], [10, 13], [11, -11], [13, -13], [11, 12], [12, 13]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 6], [5, 8], [6, 7], [6, 10], [7, 9], [8, 9], [8, 11], [9, 13], [10, 11], [10, -10], [11, 12], [12, 13], [12, -12], [13, -13]],
[[-1, 1], [-2, 2], [-4, 4], [1, 2], [1, 3], [2, 5], [3, 4], [3, 7], [4, 6], [5, 6], [5, 8], [6, 10], [7, 8], [7, 12], [8, 9], [9, 10], [9, 11], [10, 13], [11, -11], [13, -13], [11, 12], [12, 13]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 6], [5, 8], [6, 7], [6, 10], [7, 9], [8, 9], [8, 11], [9, 13], [10, 11], [10, -10], [11, 12], [12, 13], [12, -12], [13, -13]],
[[-1, 1], [-2, 2], [-4, 4], [1, 2], [1, 3], [2, 5], [3, 4], [3, 7], [4, 6], [5, 6], [5, 8], [6, 10], [7, 8], [7, 11], [8, 9], [9, 10], [9, 13], [10, 12], [11, -11], [13, -13], [11, 12], [12, 13]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [2, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 10], [7, 9], [8, 9], [8, 12], [9, 11], [10, 11], [10, 13], [11, -11], [12, 13], [12, -12], [13, -13]],
[[-1, 1], [-2, 2], [-4, 4], [1, 2], [1, 5], [2, 3], [3, 4], [3, 7], [4, 6], [5, 6], [5, 9], [6, 8], [7, 8], [7, 10], [8, 11], [9, 10], [9, 12], [10, 13], [11, -11], [13, -13], [11, 12], [12, 13]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [2, 4], [1, 5], [3, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 10], [7, 9], [8, 9], [8, 12], [9, 11], [10, 11], [10, 13], [11, -11], [12, 13], [12, -12], [13, -13]],
[[-1, 1], [-2, 2], [-4, 4], [1, 2], [1, 5], [2, 3], [3, 4], [3, 7], [4, 6], [5, 6], [5, 9], [6, 8], [7, 8], [7, 10], [8, 12], [9, 10], [9, 11], [10, 13], [11, -11], [13, -13], [11, 12], [12, 13]],
[[-1, 1], [-3, 3], [1, 2], [2, 3], [1, 4], [3, 5], [2, 7], [4, 5], [4, 8], [5, 6], [6, 7], [6, 10], [7, 9], [8, 9], [8, 12], [9, 11], [10, 11], [10, 13], [11, -11], [12, 13], [12, -12], [13, -13]],
[[-1, 1], [-2, 2], [-4, 4], [1, 2], [1, 5], [2, 3], [3, 4], [3, 7], [4, 6], [5, 6], [5, 9], [6, 8], [7, 8], [7, 10], [8, 11], [9, 10], [9, 13], [10, 12], [11, -11], [13, -13], [11, 12], [12, 13]]]
for g in Gk4:
    res = solve_lp_burl(g, 13)
    print(res.fun)
    if res.fun < 1/3:
        print(g)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.


0.6666666667026732
0.33333333334377263
0.33333333334377235
0.6666666720336585
0.666666672033659
0.6666666667026744
0.6666666666704986
0.6666666668787048
0.6666666666831915
0.6666666736447452
0.3333333333457613
0.6666666668434229
0.6666666666959945
0.6666666666959932


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: OptimizeWarning: Solving system with option 'cholesky':True failed. It is normal for this to happen occasionally, especially as the solution is approached. However, if you see this frequently, consider setting option 'cholesky' to False.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: OptimizeWarning: Solving system with option 'sym_pos':True failed. It is normal for this to happen occasionally, especially as the solution is approached. However, if you see this frequently, consider setting option 'sym_pos' to False.
/usr/local/lib/python3.7/dist-packages/scipy/optimize/_linprog_ip.py:117: LinAlgWarning: Ill-conditioned matrix (rcond=2.42145e-18): result may not be accurate.
  return sp.linalg.solve(M, r, sym_pos=sym_pos)


0.33333333333833287
0.6666666816239535
0.6666666816239504


/usr/local/lib/python3.7/dist-packages/scipy/optimize/_linprog_ip.py:117: LinAlgWarning: Ill-conditioned matrix (rcond=3.1326e-18): result may not be accurate.
  return sp.linalg.solve(M, r, sym_pos=sym_pos)


0.3333333333383296
0.6666666667129603
0.6666666667493927
0.6666666667403818
0.6666666670893574
0.33333333334227394
0.6666666667215335
0.6666666667493946
0.6666666667129653
0.6666666667215342
0.33333333334227866
0.6666666670893547
0.6666666667403792
0.6666666666692567
0.666666666714551
0.6666666667145542
0.33333333344918975
0.33333333344918725
0.6666666666692628
